In [1]:
import json
import os
import time
import torch
import torch.utils.data
import torch.optim as optim
from torchvision import transforms
from my_dataset import MyDataSet
from utils import split_data, create_lr_scheduler, get_params_groups, train_one_epoch, evaluate
from model import convnext as create_model
from torchinfo import summary

/home/lorenzo/miniconda3/envs/rfdet/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cuda:0'
data_path = "../2018.01/GOLD_XYZ_OSC.0001_1024.hdf5"
snr = 30
batch_size = 16
nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
num_classes = 24

In [3]:
device = torch.device(device if torch.cuda.is_available() else "cpu")
print(f"using {device} device...")

data_dir = os.path.dirname(os.path.abspath(data_path))
# train_indexes, train_labels, val_indexes, val_labels
reload_data = True
if os.path.exists(data_dir + '/train_indexes.json') and not reload_data:
    with open(data_dir + '/val_indexes.json') as f:
        val_indexes = json.load(f)
    with open(data_dir + '/val_label.json') as f:
        val_labels = json.load(f)
else:
    train_indexes, train_labels, val_indexes, val_labels = split_data(data_path, snr,
                                                                      ratio=[0.875, 0.125, 0.],
                                                                      test=False, one_hot=False)

print("using valid data size: {}".format(len(val_labels)))
data_transform = {
    "val": transforms.Compose([transforms.ToTensor()])
}

val_dataset = MyDataSet(hdf5_path=data_path,
                        mod_class=val_labels,
                        indexes=val_indexes,
                        transform=data_transform["val"])

using cuda:0 device...
using valid data size: 12288


In [4]:
data = val_dataset[123][0]
data = torch.unsqueeze(data, dim=0)
data.shape

torch.Size([1, 1, 1024, 2])

In [5]:
# create model
model = create_model(num_classes=num_classes).to(device)
# load model weights
model_weight_path = "./weights/best_model.pth"
model.load_state_dict(torch.load(model_weight_path, map_location=device))
model.eval()

summary(model, input_size=(batch_size, 1, 1024, 2))

Layer (type:depth-idx)                   Output Shape              Param #
ConvNeXt                                 [16, 24]                  --
├─Conv2d: 1-1                            [16, 32, 1024, 1]         96
├─ModuleList: 1-8                        --                        (recursive)
│    └─Sequential: 2-1                   [16, 32, 1024]            --
│    │    └─LayerNorm: 3-1               [16, 32, 1024]            65,536
├─ModuleList: 1-9                        --                        (recursive)
│    └─Sequential: 2-2                   [16, 32, 1024]            --
│    │    └─Block: 3-2                   [16, 32, 1024]            74,304
│    │    └─Block: 3-3                   [16, 32, 1024]            74,304
│    │    └─Block: 3-4                   [16, 32, 1024]            74,304
├─ModuleList: 1-8                        --                        (recursive)
│    └─Sequential: 2-3                   [16, 64, 256]             --
│    │    └─LayerNorm: 3-5               [

In [6]:
with torch.no_grad():
    # predict class
    output = model(data.to(device))

In [7]:
output.shape

torch.Size([1, 24])

In [8]:
output

tensor([[23.9752,  7.1303,  1.6010,  9.3841, -4.8871, -6.6282, -0.0838, -0.1094,
          2.5885, 14.3770, -1.9049,  7.3600, -2.4526,  3.9232, -7.7196, -4.7046,
         -7.8209,  9.3376,  0.0529,  1.7865,  6.8980, -2.0233, -3.0467, -6.4652]],
       device='cuda:0')

In [12]:
torch.max(output, dim=1)[1]

tensor([0], device='cuda:0')